## Exploratory Data Analysis
Let's start by cleaning and organizing the data: 

In [ ]:
import pandas as pd

# Load the data from the first sheet to inspect its structure
data = pd.read_excel('../data/CPU_Memory_Series.xlsx', sheet_name='Sheet1')

# Display the first few rows of the data to understand its format and content
data.head()


In [ ]:

# Clean the dataset by renaming columns and dropping unnecessary rows
# Assuming the first row after the header row contains valid data

# Renaming columns based on the first row
# Adjusting column renaming to match the dataset's column count
data.columns = [
    "Iteration", "CPS", "Time Start", "Time Stop", "Network Function", 
    "Cluster", "Master/Worker", "Namespace", "Microservice", "Pod",
    "Pod Age", "Pod Age Unit", "Pod Resource Limits CPU", 
    "Pod Resource Limits Memory", "Pod Resource Requests CPU",
    "Pod Resource Requests Memory", "Pod Resource Requests Factor CPU", 
    "Pod Resource Requests Factor Memory", "CPU Utilization (%)", 
    "CPU Saturation Avg (mcore)", "CPU Saturation Std", "CPU Saturation Max (mcore)",
    "CPU Saturation Min (mcore)", "Memory Utilization Avg (Mi)", 
    "Memory Utilization Std", "Memory Utilization Max (Mi)", 
    "Memory Utilization Min (Mi)", "Memory Utilization (%)",
    "Extra Column 1", "Extra Column 2"  # Placeholder names for extra columns
]

# Dropping the first row if it contains header placeholders
cleaned_data = data[1:].reset_index(drop=True)


# Converting numeric columns to appropriate data types
numeric_columns = [
    "CPS", "CPU Utilization (%)", "CPU Saturation Avg (mcore)", 
    "CPU Saturation Std", "CPU Saturation Max (mcore)", 
    "CPU Saturation Min (mcore)", "Memory Utilization Avg (Mi)", 
    "Memory Utilization Std", "Memory Utilization Max (Mi)", 
    "Memory Utilization Min (Mi)", "Memory Utilization (%)"
]
for col in numeric_columns:
    cleaned_data[col] = pd.to_numeric(cleaned_data[col], errors='coerce')

# Display cleaned data for user review
import seaborn as sns
import matplotlib.pyplot as plt

# Example: Scatter plot for CPU vs Memory Utilization
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=cleaned_data,
    x="CPU Utilization (%)",
    y="Memory Utilization Avg (Mi)",
    hue="Cluster",  # Example of adding a categorical variable for color coding
    palette="viridis",
    alpha=0.7
)

plt.title("CPU vs Memory Utilization")
plt.xlabel("CPU Utilization (%)")
plt.ylabel("Memory Utilization (Avg in Mi)")
plt.legend(title="Cluster", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()


# Statistical summary of key metrics
metrics_summary = cleaned_data[numeric_columns].describe()

# Display the summary for user review
import seaborn as sns
import matplotlib.pyplot as plt

# Transpose the summary for better visualization
metrics_summary_transposed = metrics_summary.T  # Rows become columns for heatmap

plt.figure(figsize=(12, 6))
sns.heatmap(
    metrics_summary_transposed,
    annot=True,  # Display values in each cell
    fmt=".2f",  # Format numbers to two decimal places
    cmap="coolwarm",
    cbar_kws={'label': 'Metric Value'}
)

plt.title("Metrics Summary Heatmap")
plt.xlabel("Statistics")
plt.ylabel("Metrics")
plt.tight_layout()

# Show the plot
plt.show()



In [ ]:
# Select the mean row for visualization
metrics_means = metrics_summary.loc["mean"]

plt.figure(figsize=(12, 6))
sns.barplot(
    x=metrics_means.index,
    y=metrics_means.values,
    palette="Blues_r"
)

plt.title("Mean Values of Metrics")
plt.xlabel("Metrics")
plt.ylabel("Mean Value")
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()


Let's pick one relevant microservice to focus the analysis on it: eric-ingressgw-udm-traffic 

In [ ]:
# Filter the dataset for the microservice "eric-ingressgw-udm-traffic"
selected_microservice = "eric-ingressgw-udm-traffic"
microservice_data = cleaned_data[cleaned_data["Microservice"] == selected_microservice]

# Check if data exists for this microservice
if not microservice_data.empty:
    # Display the data for the selected microservice
       
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        x=metrics_summary.columns,
        y=metrics_summary.loc["mean"],
        marker="o",
        linestyle="-",
        color="blue"
    )

    plt.title("Mean Values of Metrics")
    plt.xlabel("Metrics")
    plt.ylabel("Mean Value")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()

    # Show the plot
    plt.show()

else:
    microservice_data = None

microservice_data


Create a time series of the CPU load of the eric-ingressgw-udm-traffic microservice: 

In [ ]:
# Create a time series of CPU load for the selected microservice grouped by cluster
time_series_cpu_per_cluster = microservice_data.groupby(["Time Start", "Cluster"])["CPU Utilization (%)"].mean().reset_index()

# Sort the data for plotting
time_series_cpu_per_cluster["Time Start"] = pd.to_datetime(time_series_cpu_per_cluster["Time Start"], errors='coerce')
time_series_cpu_per_cluster = time_series_cpu_per_cluster.sort_values(by=["Time Start", "Cluster"])

# Plot the time series
plt.figure(figsize=(14, 7))
for cluster in time_series_cpu_per_cluster["Cluster"].unique():
    cluster_data = time_series_cpu_per_cluster[time_series_cpu_per_cluster["Cluster"] == cluster]
    plt.plot(
        cluster_data["Time Start"],
        cluster_data["CPU Utilization (%)"],
        label=cluster
    )

# Customize the plot
plt.title("Time Series of CPU Load for 'eric-ingressgw-udm-traffic' by Cluster")
plt.xlabel("Time Start")
plt.ylabel("CPU Utilization (%)")
plt.legend(title="Cluster", loc="upper left", bbox_to_anchor=(1.05, 1), fontsize="small")
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()


Store time CPU load time series of capo-28037-pool1-xxpmf cluster in file

In [ ]:
# Filter the time series for the specific cluster "xxpmf"
time_series_cpu_xxpmf = time_series_cpu_per_cluster[time_series_cpu_per_cluster["Cluster"] == "capo-28037-pool1-xxpmf"]

# Save the filtered time series to a CSV file
output_file_path_xxpmf = "../data/eric_ingressgw_udm_traffic_cpu_time_series_xxpmf.csv"
time_series_cpu_xxpmf.to_csv(output_file_path_xxpmf, index=False)

output_file_path_xxpmf


Now for cluster "lk5nv": 

In [ ]:
# Filter the time series for the specific cluster "xxpmf"
time_series_cpu_lk5nv = time_series_cpu_per_cluster[time_series_cpu_per_cluster["Cluster"] == "capo-28037-pool1-lk5nv"]

# Save the filtered time series to a CSV file
output_file_path_lk5nv = "../data/eric_ingressgw_udm_traffic_cpu_time_series_lk5nv.csv"
time_series_cpu_lk5nv.to_csv(output_file_path_lk5nv, index=False)

output_file_path_lk5nv